In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from keras.utils import np_utils

In [ ]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/fer2013.csv')
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [ ]:
df.shape

(35887, 3)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


In [ ]:
emotion_labels = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}
Labels = [1,3,4,6]
df = df[df.emotion.isin(Labels)]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21811 entries, 3 to 35885
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  21811 non-null  int64 
 1   pixels   21811 non-null  object
 2   Usage    21811 non-null  object
dtypes: int64(1), object(2)
memory usage: 681.6+ KB


In [ ]:
img_array = df.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48, 1).astype('float32'))
img_array

3        [[[24.0], [32.0], [36.0], [30.0], [32.0], [23....
4        [[[4.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...
6        [[[20.0], [17.0], [19.0], [21.0], [25.0], [38....
7        [[[77.0], [78.0], [79.0], [79.0], [78.0], [75....
8        [[[85.0], [84.0], [90.0], [121.0], [101.0], [1...
                               ...                        
35877    [[[139.0], [143.0], [145.0], [154.0], [159.0],...
35878    [[[0.0], [39.0], [81.0], [80.0], [104.0], [97....
35882    [[[50.0], [36.0], [17.0], [22.0], [23.0], [29....
35883    [[[178.0], [174.0], [172.0], [173.0], [181.0],...
35885    [[[30.0], [28.0], [28.0], [29.0], [31.0], [30....
Name: pixels, Length: 21811, dtype: object

In [ ]:
img_array = np.stack(img_array, axis=0)
img_array

array([[[[ 24.],
         [ 32.],
         [ 36.],
         ...,
         [173.],
         [172.],
         [173.]],

        [[ 25.],
         [ 34.],
         [ 29.],
         ...,
         [173.],
         [172.],
         [173.]],

        [[ 26.],
         [ 29.],
         [ 25.],
         ...,
         [172.],
         [172.],
         [174.]],

        ...,

        [[159.],
         [185.],
         [157.],
         ...,
         [157.],
         [156.],
         [153.]],

        [[136.],
         [157.],
         [187.],
         ...,
         [152.],
         [152.],
         [150.]],

        [[145.],
         [130.],
         [161.],
         ...,
         [142.],
         [143.],
         [142.]]],


       [[[  4.],
         [  0.],
         [  0.],
         ...,
         [ 27.],
         [ 24.],
         [ 25.]],

        [[  1.],
         [  0.],
         [  0.],
         ...,
         [ 26.],
         [ 23.],
         [ 26.]],

        [[  0.],
         [  0.],
      

In [ ]:
LE = LabelEncoder()
img_labels = LE.fit_transform(df['emotion'])
img_labels = np_utils.to_categorical(img_labels)
img_labels.shape

(21811, 4)

In [ ]:
img_labels

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [ ]:
LE_label_mapping = dict(zip(LE.classes_, LE.transform(LE.classes_)))

In [ ]:
LE_label_mapping

{1: 0, 3: 1, 4: 2, 6: 3}

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(img_array, img_labels,
                                                    shuffle=True, stratify=img_labels,
                                                    test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((17448, 48, 48, 1), (4363, 48, 48, 1), (17448, 4), (4363, 4))

In [ ]:
img_width = X_train.shape[1]
img_height = X_train.shape[2]
img_depth = X_train.shape[3]
num_classes = y_train.shape[1]

In [ ]:
X_train = X_train / 255.
X_test = X_test / 255.

In [ ]:
def build_net(optim):
            net = Sequential(name='DCNN')

            net.add(Conv2D(filters=256,kernel_size=(5,5),input_shape=(img_width, img_height, img_depth),activation='elu',
                           padding='same',kernel_initializer='he_normal', name='conv2d_1'))
            net.add(BatchNormalization(name='batchnorm_1'))
            net.add(Conv2D(filters=128,kernel_size=(5,5),activation='elu',padding='same',kernel_initializer='he_normal',
                        name='conv2d_2'))
            net.add(BatchNormalization(name='batchnorm_2'))
            net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_1'))
            net.add(Dropout(0.4, name='dropout_1'))
            net.add(Conv2D(filters=128,kernel_size=(3,3),activation='elu',padding='same',kernel_initializer='he_normal',
                        name='conv2d_3'))
            net.add(BatchNormalization(name='batchnorm_3'))
            net.add(Conv2D(filters=128,kernel_size=(3,3),activation='elu',padding='same',kernel_initializer='he_normal',
                        name='conv2d_4'))
            net.add(BatchNormalization(name='batchnorm_4'))
            net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_2'))
            net.add(Dropout(0.4, name='dropout_2'))
            net.add(Conv2D(filters=256,kernel_size=(3,3),activation='elu',padding='same',kernel_initializer='he_normal',
                        name='conv2d_5'))
            net.add(BatchNormalization(name='batchnorm_5'))
            net.add(Conv2D(filters=256,kernel_size=(3,3),activation='elu',padding='same',kernel_initializer='he_normal',
                        name='conv2d_6'))
            net.add(BatchNormalization(name='batchnorm_6'))
            net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_3'))
            net.add(Dropout(0.5, name='dropout_3'))
            net.add(Flatten(name='flatten'))
            net.add(Dense(128,activation='elu',kernel_initializer='he_normal',name='dense_1'))
            net.add(BatchNormalization(name='batchnorm_7'))
            net.add(Dropout(0.6, name='dropout_4'))
            net.add(Dense(num_classes,activation='softmax',name='out_layer'))

            net.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
            net.summary()
            
            return net

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy',min_delta=0.00005,patience=10,verbose=1,
    restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy',factor=0.5,patience=7,min_lr=1e-7,verbose=1)

callbacks = [early_stopping,lr_scheduler]

In [ ]:
train_datagenerator = ImageDataGenerator(rotation_range=15,width_shift_range=0.15,height_shift_range=0.15,
    shear_range=0.15,zoom_range=0.15,horizontal_flip=True)
train_datagenerator.fit(X_train)

In [ ]:
batch_size = 64
epochs = 75
optims = [optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam')
          ,optimizers.Adam(0.001)]

model = build_net(optims[1]) 
history = model.fit(train_datagenerator.flow(X_train, y_train, batch_size=batch_size),validation_data=(X_test, y_test),
    steps_per_epoch=len(X_train) / batch_size,epochs=epochs,callbacks=callbacks)

Model: "DCNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 48, 48, 256)       6656      
                                                                 
 batchnorm_1 (BatchNormaliza  (None, 48, 48, 256)      1024      
 tion)                                                           
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 128)       819328    
                                                                 
 batchnorm_2 (BatchNormaliza  (None, 48, 48, 128)      512       
 tion)                                                           
                                                                 
 maxpool2d_1 (MaxPooling2D)  (None, 24, 24, 128)       0         
                                                                 
 dropout_1 (Dropout)         (None, 24, 24, 128)       0      

In [ ]:
model.save("/content/drive/MyDrive/Dataset/emote_model.h5")

In [ ]:
model.save("/content/drive/MyDrive/Dataset/emote_model.pkl")

In [ ]:
import keras.models
keras_file = "/content/drive/MyDrive/Dataset/emote_model.h5"
keras.models.save_model(model, keras_file)

In [ ]:
import tensorflow as tf
tflite_model = tf.keras.models.load_model('/content/drive/MyDrive/Dataset/emote_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(tflite_model)
tflite_save = converter.convert()
open("/content/drive/MyDrive/Dataset/emote_model.tflite","wb").write(tflite_save)

12765696